### HTML 문서 읽기 & 데이터 값 찾기

* 문서 URL 목록 가져오기

In [7]:
# 금액 단위 찾기
def get_unit(text):
    if '(단위:백만원)' in text or '[단위:백만원]' in text:
        unit = '백만원'
    elif '(단위:천원)' in text or '[단위:천원]' in text:
        unit = '천원'
    elif '(단위:원)' in text or '[단위:원]' in text:
        unit = '원'
    else:
        unit = None   
    return unit

# 콤마와 괄호를 제거한다.
def str2num(x):
    if "," in x:
        x = x.replace(",","")
    if x.startswith("("):
        x = x.lstrip("(")
    if x.endswith(")"):
        x = x.rstrip(")")
    return float(x)

In [9]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

# 파일 저장 위치
doc_url = 'http://dart.fss.or.kr/report/viewer.do?rcpNo=20230315001359&dcmNo=9061217&eleId=22&offset=893845&length=450250&dtd=dart3.xsd'

# 재무제표 주석 문서에서 사용된 금액 단위 찾기
response = requests.get('doc_url')
if response.status_code == 200:     # URL GET '200 정상'
    soup = BeautifulSoup(response.text)
    doc_text = soup.text
    trimmed_text = doc_text.replace(" ","")     # 문서에 있는 모든 공백 제거
    unit = get_unit(trimmed_text)
else:
    unit = None
# 당기말의 "확정급여채무"와 "사외적립자산" 금액 찾기
doc_data = pd.read_html(response.text)
for df in doc_data:
    row = df.shape[0]
    for i in range(0, row):
        item = str(df.iloc[i,0])
        if item.__contains__('확정급여채무의 현재가치'):
            # print(i, type(df.iloc[i,1]), df.iloc[i,1])
            str_a = str(df.iloc[i,1])
            if str_a == "-": val_a = 0.0
            else: val_a += str2num(str_a)       # 값을 합한다.
        elif item.__contains__('사외적립자산의 공정가치'):
            # print(i, type(df.iloc[i,1]), df.iloc[i,1])                            
            str_b = str(df.iloc[i,1])
            if str_b == "-": val_b = 0.0
            else: val_b = str2num(str_b)
        else:
            continue    
    if (val_a != 0.0) and (val_b != 0.0):
        break          
print(f'단위: {unit}, 확정급여채무의 현재가치: {val_a}, 사외적립자산의 공정가치: {val_b}')

ImportError: cannot import name 'HTMLParserTreeBuilder' from 'bs4.builder' (d:\PythonProject\data-gatherer\data-gatherer.venv\lib\site-packages\bs4\builder\__init__.py)